In [ ]:
from bs4 import BeautifulSoup

path = 'produced_data/body.xml'

with open(path, 'r', encoding='utf-8') as f:
    def retrieve_person(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        persons = soup.find_all('person')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(person['xml:id'], person.find('persName').text) for person in persons]

        return xml_ids_and_value_labels
    
    def retrieve_places(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        places = soup.find_all('place')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [{place.find('placeName').text: place['xml:id']} for place in places]

        return xml_ids_and_value_labels

# Example usage:

    places = retrieve_places(f)

    print(places)

In [ ]:
from labelList import personList, placeList
from worklocationdict import worklocationdict
import sparql_dataframe
import pandas
from ratelimit import limits

#store the people missing from wikidata and the multiple entries
missingpeople=[]
multiplepeople=[]

# placeListLabel = list(key for dictionary in placeList for key in dictionary)       
placeListLabel = set(placeList.keys())

@limits(calls=1, period=61)
def personRDF():
       result=[]
       for x,y in personList:        
              
              name = y

              endpoint = 'https://query.wikidata.org/sparql'
              
              queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel
              
              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@it.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "it")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

              df_first = sparql_dataframe.get(endpoint, queryPerson, post='True')
              df = df_first.fillna('')


              #check if there's only one person in wikidata
              
              same = df['human'].unique()
              
              if df.empty:
                     endpoint = 'https://query.wikidata.org/sparql'

                     queryPerson = """
                     SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

                     WHERE {
                     ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                            rdfs:label "%s"@en.  # Label in Italian
                     ?human rdfs:label ?humanLabel.  # Get the label
                     FILTER(LANG(?humanLabel) = "en")  # Filter for Italian labels

                     OPTIONAL {?human wdt:P21 ?gender.
                            ?gender rdfs:label ?genderLabel.
                            FILTER(LANG(?genderLabel) = "it")}
                     OPTIONAL{?human wdt:P569 ?birthDate.}
                     OPTIONAL{?human wdt:P19 ?birthPlace.
                            ?birthPlace rdfs:label ?birthPlaceLabel.
                            FILTER(LANG(?birthPlaceLabel) = "it")}
                     OPTIONAL{?human wdt:P570 ?deathDate.}
                     OPTIONAL{?human wdt:P20 ?deathPlace.
                            ?deathPlace rdfs:label ?deathPlaceLabel.
                            FILTER(LANG(?deathPlaceLabel) = "it") }
                     OPTIONAL{?human wdt:P106 ?occupation.
                            ?occupation rdfs:label ?occupationLabel.
                            FILTER(LANG(?occupationLabel) = "it")}
                     OPTIONAL {?human wdt:P937 ?workLocation.
                            ?workLocation rdfs:label ?workLocationLabel.
                            FILTER(LANG(?workLocationLabel) = "it")}

                     }
                     """%(name)

              df_first = sparql_dataframe.get(endpoint, queryPerson, post='True')
              df = df_first.fillna('')

              same=df['human'].unique() 

              if df.empty:

                     missingpeople.append((x,y))

              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)


              else:
                     birthPlaceLabel = df['birthPlaceLabel'][0] 
                     deathPlaceLabel = df['deathPlaceLabel'][0]
                     createrdf = """\n<rdf:Description tei:ref='{id}' rdf:about='{wikidataentity}'>\n\t<rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>\n\t<rdfs:label xml:lan="it">{label}</rdfs:label>\n\t<crm:P61_has_gender>{genderLabel}</crm:P61_has_gender>\n\t<schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{birthDate}</schema:birthDate>\n\t<schema:deathDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{deathDate}</schema:deathDate>\n""".format(id=x, wikidataentity=df['human'][0], label=df['humanLabel'][0], genderLabel=df['genderLabel'][0], birthDate=df['birthDate'][0], deathDate=df['deathDate'][0])
                     
              #add birth place as referenced in the header or as entity in wikidata
              
                     if birthPlaceLabel in placeList.keys():
                     
                            birthPlaceLabelId= placeList[birthPlaceLabel]

                            createrdf = createrdf + '\t<schema:birthPlace xml:ref="%s">%s</schema:birthPlace>\n'%(birthPlaceLabelId, birthPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:birthPlace rdf:resource="%s">%s<schema:birthPlace>\n'%(df['birthPlace'][0], df['birthPlaceLabel'][0])

              # #add death place as referenced in the header or as entity in wikidata

                     if deathPlaceLabel in placeList.keys():

                            deathPlaceLabelId = placeList[deathPlaceLabel]

              # #ho dei dubbi sul primo caso
                            createrdf = createrdf + '\t<schema:deathPlace xml:ref="%s">%s</schema:deathPlace>\n'%(deathPlaceLabelId, deathPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="%s">%s</schema:deathPlace>\n'%(df['deathPlace'][0], df['deathPlaceLabel'][0])

                     if df['occupation'][0] != '':
                            for occupation in df['occupation'].unique():
                                   occupationLabel = df.loc[df['occupation'] == occupation, 'occupationLabel'].values[0]
                                   createrdf = createrdf + '\t<schema:hasOccupation rdf:resource="%s">%s</schema:hasOccupation>\n'%(occupation, occupationLabel)
                     
                     #if there is at least one workLocation
                     if df['workLocation'][0] != '':
                            #if the person is an artist
                            if df['humanLabel'][0] in worklocationdict.keys():
                                   # for i in worklocationdict[df['humanLabel'][0]]:
                                   #        if i in placeListLabel:

                                   #find all the places named by the book that wikidata consider a workLocation of her
                                   worklocationinwdt = set(df['workLocationLabel'].unique())
                                   worklocationinwdtandbook = worklocationinwdt.intersection(placeListLabel)
                                   for workLocationLabel in list(worklocationinwdtandbook):
                                          # if workLocationLabel in placeList.keys():
                                          workLocationId= placeList[workLocationLabel]
                                          createrdf = createrdf + '\t<schema:workLocation xml:ref="%s">%s</schema:workLocation>\n'%(workLocationId, workLocationLabel)

                                          # else:
                                          #        createrdf = createrdf + '\t<schema:birthPlace rdf:resource="%s">%s<schema:birthPlace>\n'%(df['workLocation'][0], worklocationlabel)


                                          #        worklocationlabel = df.loc[df['workLocationLabel'] == i, 'workLocation'].values[0]
                                          #        createrdf = createrdf + '\t<schema:workLocation rdf:resource"%s">%s</schema:workLocation>\n'%(worklocationlabel, i)
                            
                     
                     
                     
                     createrdf = createrdf + "</rdf:Description>\n"
                     print(createrdf)

# Execution
rdf = personRDF()
print(rdf)
print(missingpeople, len(missingpeople))
print(multiplepeople, len(multiplepeople))


In [ ]:
#query for places

import sparql_dataframe
from labelList import placeList4, placeList2, placeList3, placeList
from ratelimit import limits

missingplace=[]
multipleplace=[]

@limits(calls=1, period=61)
def placeRDF(placeList):
       result = []
       for i in list(placeList.keys()):

              placelabel = i
              placelabelid = placeList[i]
              

              endpoint = 'https://query.wikidata.org/sparql'

              queryPlace= """SELECT ?place ?placeLabel ?coordinate  WHERE {
              ?place wdt:P31/wdt:P279* wd:Q618123; # Instances of place (Q2221906) or its subclasses
                     rdfs:label "%s"@it; # Label in English
                     wdt:P625 ?coordinate.
              ?place rdfs:label ?placeLabel. # Get the label
              FILTER(LANG(?placeLabel) = "it") # Filter for English labels
              }
              """%placelabel
              
              df = sparql_dataframe.get(endpoint, queryPlace, post='True')
              
              same = df['place'].unique()

              if df.empty:
                     missingplace.append(i)
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multipleplace.append(multiple_entries)
              
              else:

                     coordinate = df['coordinate'][0]
                     cleancoordinate = coordinate[6:-1]
                     finalcoordinate = cleancoordinate.split(' ')

                     createrdf ="""
              <rdf:Description tei:ref="{id}" rdf:about="{about}">
                     <rdf:type rdf:resource="http://www.cidoc-crm.org/cidoc-crm/7.1.1/E_53_Place"/>
                     <rdfs:label xml:lang="it">{label}</rdfs:label>
                     <geo:lat>{lat}</geo:lat>
                     <geo:long>{long}</geo:long>
              </rdf:Description>

              """.format(id=placelabelid, wid = df['place'][0], label = placelabel, lat = finalcoordinate[1], long = finalcoordinate[0], about = df['place'][0])
       
                     result.append(createrdf)
                     print(createrdf)
       
       return result 

rdf = placeRDF(placeList4)
print(rdf)
print(missingplace, len(missingplace))
print(multipleplace, len(multipleplace))

In [32]:
from labelList import workList, personList, placeList
import sparql_dataframe
from ratelimit import limits

@limits(calls=1, period=122)
def workRDF():
    result = []

    for item in list(workList.keys()):
        
        endpoint = 'https://query.wikidata.org/sparql'
        
        query = """select distinct ?entity ?entityLabel ?author ?authorLabel ?label ?title ?type ?typeLabel ?inception ?creationLocation ?creationLocationLabel ?creationLocationCoordinate ?depicts ?depictsLabel ?movement ?movementLabel ?genre ?genreLabel ?location ?locationLabel 
    ?locationCoordinate ?city ?cityLabel where{
        BIND (wd:%s AS ?entity)
        ?entity wdt:P170 ?author ;
                    rdfs:label ?label ;
                    wdt:P31 ?type .
        ?author rdfs:label ?authorLabel .
                ?type rdfs:label ?typeLabel.
                FILTER(LANG(?authorLabel) = "it")
                 FILTER(LANG(?label) = "it")
                FILTER(LANG(?typeLabel) = "it")
        optional{?entity wdt:P1476 ?title .}
        optional{?entity wdt:P135 ?movement.
                ?movement rdfs:label ?movementLabel.
                FILTER(LANG(?movementLabel) = "it")}
        optional{?entity wdt:P136 ?genre.
                ?genre rdfs:label ?genreLabel.
                FILTER(LANG(?genreLabel) = "it")}           
        optional{?entity wdt:P571 ?inception .}      
        optional{?entity wdt:P180 ?depicts .
                ?depicts rdfs:label ?depictsLabel.
                FILTER(LANG(?depictsLabel) = "it")} 
        optional{?entity wdt:P276 ?location.
                ?location rdfs:label ?locationLabel.
                FILTER(LANG(?locationLabel) = "it")
                optional{?location wdt:P625 ?locationCoordinate.}
                optional{?location wd:P131 ?city.
                        ?city rdfs:label ?cityLabel.
                        FILTER(LANG(?cityLabel) = "it")}}
        optional{?entity wdt:P1071 ?creationLocation.
                ?creationLocation rdfs:label ?creationLocationLabel.
                FILTER(LANG(?creationLocationLabel) = "it")
                    optional{?creationLocation wdt:P625 ?creationLocationCoordinate. }}
        }"""%item

        df_first = sparql_dataframe.get(endpoint, query, post='True')
        df = df_first.fillna("")


    #     if df.empty:

    #         id = 'wd:Q20810265'#%s'%item
        
    #         endpoint = 'https://query.wikidata.org/sparql'
        
    #         query = """select distinct ?entity ?entityLabel ?author ?authorLabel ?label ?title ?type ?typeLabel ?inception ?creationLocation ?creationLocationLabel ?creationLocationCoordinate ?depicts ?depictsLabel ?movement ?movementLabel ?genre ?genreLabel ?location ?locationLabel 
    # ?locationCoordinate ?city ?cityLabel where{
    #     BIND (%s AS ?entity)
    #     ?entity wdt:P170 ?author;
    #                 rdfs:label ?label;
    #                 wdt:P31 ?type.
    #             ?author rdfs:label ?authorLabel.
    #             ?type rdfs:label ?typeLabel.
    #             FILTER(LANG(?authorLabel) = "it")
    #             FILTER(LANG(?label) = "en")
    #             FILTER(LANG(?typeLabel) = "it")
    #     optional{?entity wdt:P1476 ?title. }
    #     optional{?entity wdt:P135 ?movement.
    #             ?movement rdfs:label ?movementLabel.
    #             FILTER(LANG(?movementLabel) = "it")}
    #     optional{?entity wdt:P136 ?genre.
    #             ?genre rdfs:label ?genreLabel.
    #             FILTER(LANG(?genreLabel) = "it")}           
    #     optional{?entity wdt:P571 ?inception .}      
    #     optional{?entity wdt:P180 ?depicts .
    #             ?depicts rdfs:label ?depictsLabel.
    #             FILTER(LANG(?depictsLabel) = "it")} 
    #     optional{?entity wdt:P276 ?location.
    #             ?location rdfs:label ?locationLabel.
    #             FILTER(LANG(?locationLabel) = "it")
    #             optional{?location wdt:P625 ?locationCoordinate.}
    #             optional{?location wd:P131 ?city.
    #                     ?city rdfs:label ?cityLabel.
    #                     FILTER(LANG(?cityLabel) = "it")}}
    #     optional{?entity wdt:P1071 ?creationLocation.
    #             ?creationLocation rdfs:label ?creationLocationLabel.
    #             FILTER(LANG(?creationLocationLabel) = "it")
    #                 optional{?creationLocation wdt:P625 ?creationLocationCoordinate. }}
    #     }"""%(id)

    #         df_first = sparql_dataframe.get(endpoint, query, post='True')
    #         df_en = df_first.fillna("")

    #         print(df_en)

            

        # print(df)
        if not df.empty:
            createrdf = '\n<rdf:Description tei:ref="{ref}" rdf:about="{about}">\n\t<rdfs:label xml:lan="it">{label}</rdfs:label>\n\t<rdf:type rdf:resource="{type}">{typeLabel}</rdf:type>\n\t'.format(ref=workList[item], about=df['entity'][0], label = df['label'][0], type=df['type'][0], typeLabel= df['typeLabel'][0])
            for x,y in personList:    
                if df['authorLabel'][0] == y:
                    createrdf = createrdf + '<crm:P_108_was_produced_by tei:ref="{personref}">{authorLabel}</crm:P_108_was_produced_by>\n\t'.format(personref= x,authorLabel =y)

            if df['inception'][0] != '':
                createrdf = createrdf + '<crm:E_52_Time_Span>%s</crm:E_52_Time_Span>\n\t'%df['inception'][0]
            
            if df['depicts'][0] != '':
                for i in df['depicts'].unique():
                    depictsLabel =df.loc[df['depicts'] == i, 'depictsLabel'].values[0]         
                    createrdf = createrdf + '<crm:P_138_represents rdf:about="%s">%s</crm:P_138_represents>\n\t'%(i, depictsLabel)
                    
            if df['movement'][0] != '':
                for i in df['movement'].unique():
                    movementLabel =df.loc[df['movement'] == i, 'movementLabel'].values[0]         
                    createrdf = createrdf + '<crm:P_2_has_type rdf:about="%s">%s</crm:P_2_has_type>\n\t'%(i, movementLabel)
                    
            if df['genre'][0] != '':
                for i in df['genre'].unique():
                    genreLabel =df.loc[df['genre'] == i, 'genreLabel'].values[0]         
                    createrdf = createrdf + '<crm:P129_is_about rdf:about="%s">%s</crm:P_129_is_about>\n\t'%(i, genreLabel)

                
            creationLocationLabel = df['creationLocationLabel'][0]
                
            if df['locationLabel'][0] != '':
                locationLabel = df['locationLabel'][0]
                
                location = df.loc[df['locationLabel'] == locationLabel, 'location'].values[0]  
                #add birth place as referenced in the header or as entity in wikidata
                
                if locationLabel in placeList:
                    locationId = placeList[locationLabel]           
                    
                    createrdf = createrdf + '<schema:location xml:ref="%s">%s</schema:location>\n'%(locationId, locationLabel)
                    

                else:
                    createrdf = createrdf + '<schema:location rdf:resource="%s">%s</schema:location>\n'%(location, locationLabel)

                
                if df['creationLocationLabel'][0] != '':
                    creationLocationLabel = df['creationLocationLabel'][0]
                
                    if creationLocationLabel in placeList.keys():
                    
                        creationLocationId= placeList[creationLocationLabel]
                        creationLocation = df.loc[df['creationLocationLabel'] == creationLocationLabel, 'creationLocation'].values[0]  

                        createrdf = createrdf + '<schema:locationCreated xml:ref="%s">%s</schema:locationCreated>\n'%(creationLocationId, creationLocation)

                    else:  
                        creationLocationId= placeList[creationLocationLabel]
                        creationLocation = df.loc[df['creationLocationLabel'] == creationLocationLabel, 'creationLocation'].values[0]  
                    
                        createrdf = createrdf + '<schema:locationCreated rdf:resource="%s">%s</schema:locationCreated>\n'%(creationLocation, creationLocationLabel)
                    
        createrdf = createrdf + '</rdf:Description>'
        
    
        print(createrdf)

workRDF()

SyntaxError: invalid syntax (3418677919.py, line 140)